# Causal-IR-DIR áp dụng trên McMaster dataset bằng $\text{DIR}_{\text{sf}}$

In [4]:
import os
from zipfile import ZipFile

import cv2
import numpy as np

import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
import torch.optim as optim
from torch.utils import data

c:\Users\C15TQK\anaconda3\envs\uit-cs431\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set-up về việc có dùng Google Colab hay không

Nếu đang chạy trên Google Colab thì path đến file ZIP của dataset sẽ được cập nhật theo shortcut trên Google Drive. (Xem thêm trong file `README.md`)

In [3]:
is_gcolab: bool = False
try:
    if str(get_ipython()) == 'google.colab':
        is_gcolab = True
except NameError:
    pass

if is_gcolab:
    print('This notebook is running on Google Colab')
else:
    print('This notebook is not running on Google Colab')

This notebook is not running on Google Colab


In [14]:
if is_gcolab:
    from google.colab import drive
    drive.mount('/gdrive')
    print('Mounted from Google Drive')
else:
    print('Because this is not running with Google Colab, '+
          'data will not be mounted from Google Drive')

Because this is not running with Google Colab, data will not be mounted from Google Drive


## Load dữ liệu

McMaster dataset

In [13]:
# Set the path to ZIP file

data_zip_filepath = None
if is_gcolab:
    data_zip_filepath = '/gdrive/MyDrive/McM.zip'
else:
    data_zip_filepath = './McM.zip'

assert data_zip_filepath is not None
assert os.path.exists(data_zip_filepath)

print(data_zip_filepath)

./McM.zip


In [15]:
# Extract all from ZIP file

data_dirpath = './McM'

with ZipFile(data_zip_filepath, 'r') as zip_ref:
    zip_ref.extractall(data_dirpath, pwd='McM_CDM'.encode('utf-8'))

assert data_dirpath is not None
assert os.path.exists(data_dirpath)

print(data_dirpath)

./McM


## Tiền xử lý

### Cắt ảnh ra theo patch 256x256

In [20]:
# Causal-IR-DIL/generate_cropped_DF2K.py

folder = os.path.join(data_dirpath, 'McM')
dest_folder = './McM-cropped'

if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

patchsize = 256
stride = 256

count = 1

for img_n in sorted(os.listdir(folder)):
    img = cv2.imread(os.path.join(folder, img_n))
    h, w, _ = img.shape
    h_number = h // patchsize
    w_number = w // patchsize
    for i in range(h_number):
        for j in range(w_number):
            start_ij_l = j * stride
            start_ij_u = i * stride
            end_ij_l = start_ij_l + stride
            end_ij_u = start_ij_u + stride
            img_crop = img[start_ij_u:end_ij_u, start_ij_l:end_ij_l]
            cv2.imwrite(os.path.join(dest_folder, '{:0>6d}.png'.format(count)), img_crop)
            count += 1
print("{} done!".format(folder))

./McM\McM done!


## Xây dựng mô hình sử dụng $\text{DIR}_{\text{sf}}$

## Huấn luyện


## Đánh giá